In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import sys

address = 'CDG-for-jupyter-notebooks/LLMSamples'
sys.path.append(f'content/drive/MyDrive/{address}')

%cd /content/drive/MyDrive/$address
%pwd

/content/drive/MyDrive/CDG-for-jupyter-notebooks/LLMSamples


'/content/drive/MyDrive/CDG-for-jupyter-notebooks/LLMSamples'

In [ ]:
!pip install transformers
!pip install accelerate
!pip install git+https://github.com/ISE-Code-Documentation-Generators/utility.git
!pip install git+https://github.com/ISE-Code-Documentation-Generators/data.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
from typing import Dict
import os
import json

import torch, random, pandas as pd
from transformers import pipeline
from ise_cdg_utility.metrics import NLPMetricInterface, CodeMetric, get_metrics
from ise_cdg_data.tokenize import get_source_and_markdown_tokenizers

In [ ]:
random.seed(4444)

In [ ]:
dataset_path = "samples_dataset.csv"
dataset = pd.read_csv(dataset_path)

def dataset_get_markdown(index: int, project_id=None):
    df = dataset
    if project_id is not None:
        df = df[df['project_ID']==project_id]
    return str(df.iloc[index]['markdown'])

def dataset_get_source(index: int, project_id=None):
    df = dataset
    if project_id is not None:
        df = df[df['project_ID']==project_id]
    return str(df.iloc[index]['source'])

In [ ]:
class Pipeline:
    def __init__(self, l):
        self.l = l

    def to_map(self, f):
        return Pipeline(list(map(f, self.l)))

    def to_reduce(self, f, initial=None):
        from functools import reduce
        return reduce(f, self.l, initial)

    def to_list(self):
        return self.l

class Template:
    def __init__(self, index: int, row_index: int, project_id=None):
        self.index = index
        self.row_index = row_index
        self.project_id = project_id

    def representative_index(template):
        return str(template.index + 1)

    def generate_prompt(template):
        return (f"#Code\n{dataset_get_markdown(template.row_index)}\n"
                + f"#Summary: {dataset_get_source(template.row_index)}\n"
                )

class Sample:
    def __init__(self, template_indices, question_index, project_id=None):
        self.templates = (Pipeline(range(len(template_indices)))
                          .to_map(lambda template_index: Template(template_index, template_indices[template_index], project_id=project_id))
                          .to_list())
        self.question_index = question_index
        self.project_id = project_id

    def generate_prompt(self):
        return ("You are an expert Python programmer, please describe the functionality of the method:\n"
                + "".join(Pipeline(self.templates)
                           .to_map(lambda template: template.generate_prompt())
                           .to_list())
                + f"\n#Code\n{dataset_get_source(self.question_index)}\n#Summary:"
               )

    def get_ground_truth(self):
        return dataset_get_markdown(self.question_index)

def generate_prompt_data(samples):
    prompt_list = Pipeline(samples).to_map(lambda sample: sample.generate_prompt()).to_list()
    grund_truth = Pipeline(samples).to_map(lambda sample: sample.get_ground_truth()).to_list()
    return prompt_list, grund_truth

In [ ]:
sample_size = 10
shot_size = 5

def get_samples():
    df = dataset
    samples = []
    for pid in df['project_ID'].unique():
        qid = qid = random.sample(range(0, len(df[df['project_ID']==pid])), 1)[0]
        templates = [
            ind
            for ind in random.sample(
                range(0, dataset.shape[0]
                    ), min(shot_size, len(df[df['project_ID']==pid])))
            if ind != qid
            ]
        samples.append(Sample(
            template_indices=templates, question_index=qid,
        ))
    return samples

prompt_list, ground_truth = generate_prompt_data(get_samples())

In [ ]:
dolly = pipeline(model="databricks/dolly-v2-3b",
                            torch_dtype=torch.bfloat16,
                            trust_remote_code=True,
                            device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/819 [00:00<?, ?B/s]

instruct_pipeline.py:   0%|          | 0.00/9.16k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/databricks/dolly-v2-3b:
- instruct_pipeline.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/5.68G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/450 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def prompt_model(input):
  dolly_response = dolly(input, max_new_tokens=100)
  return dolly_response[0]["generated_text"]

In [ ]:
metrics: Dict[CodeMetric, NLPMetricInterface] = get_metrics()
_, md_tokenizer = get_source_and_markdown_tokenizers(cleanse_markdown=False)

In [ ]:
references = []
for i in range(len(ground_truth)):
  gt = ground_truth[i]
  reference = md_tokenizer(gt)
  references.append([reference])

print(references)

[[['left', 'the', 'kids', 'downstairs', '.', 'parents', 'that', 'have', 'much', 'better', 'room', 'than', 'their', 'children', '.']], [['party', 'all', 'night', 'third', 'class', 'and', 'alone', 'and', 'spent', 'less', 'than', '5', 'dollars']], [['duffing', 'map']], [['what', 'a', 'coincidence', 'travelling', 'with', 'a', 'woman', 'and', 'she', "'s", 'not', 'his', 'wife', '.']], [['nothing', 'but', 'the', 'best', 'a', 'person', 'that', 'paid', 'an', 'unusually', 'high', 'price', '(', 'z', '-', 'score', ')', 'for', 'his', 'ticket', '.']], [['nanny', 'in', 'every', 'corner', 'travelling', 'first', 'class', 'and', 'have', 'at', 'least', 'one', 'nanny']], [['stole', 'the', 'ticket', 'spent', 'less', 'than', '5', 'dollars']], [['is', 'rich', 'travelling', 'first', 'class', 'and', 'alone', 'or', 'with', 'a', 'nanny', 'and', 'spent', 'more', 'than', '40', 'dollars']], [['party', 'all', 'night', 'third', 'class', 'and', 'alone', 'and', 'spent', 'less', 'than', '5', 'dollars']], [['who', 'the',

In [ ]:
for metric in metrics.values():
  metric.set_references(references)

In [ ]:
candidates = []
for i in range(len(prompt_list)):
  prompt = prompt_list[i]
  candidate = md_tokenizer(prompt_model(prompt))
  candidates.append(candidate)

print(candidates)

[['the', 'area', 'of', 'a', 'circle', 'is', 'equal', 'to', 'pi*(radius)^2', '.', 'the', 'radius', 'is', 'the', 'value', 'of', 'the', 'diameter', 'divided', 'by', '2', '.', 'round', 'the', 'value', 'of', 'the', 'area', 'to', 'two', 'decimal', 'places', '.', 'the', 'area', 'of', 'a', 'circle', 'is', 'equal', 'to', 'pi', '*', '(', 'd/2)^2', '.'], ['please', 'note', 'that', 'this', 'example', 'uses', 'numpy', ',', 'the', 'python', 'library', 'for', 'numerical', 'computation', ',', 'as', 'well', 'as', 'pyplot', '.', 'also', ',', 'note', 'that', 'both', 'a', 'and', 'b', 'are', 'lists', ',', 'and', 'the', 'indexing', 'of', 'each', 'element', 'can', 'be', 'performed', 'with', 'the', 'enumerate', '(', ')', 'function', '.'], ['this', 'just', 'takes', 'the', 'mean', 'and', 'standard', 'deviation', 'of', 'a', 'dataset', 'and', 'scales', 'the', 'data', 'to', 'fall', 'in', 'that', 'range'], ['windmi', 'attractor', '\\begin{alignedat}{0', '}', '\\frac{dx}{dt', '}', '=', 'y\\', '\\', '\\frac{dy}{dt', 

In [ ]:
results = {}
for metric_name, metric in metrics.items():
  uncleaned_result = metric(candidates)
  result = {}
  for k, v in uncleaned_result.items():
    result[k] = float(v)
  results[metric_name.value] = result

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
results

{'bleu': {'bleu_1': 0.06927710771560669,
  'bleu_2': 0.020743494853377342,
  'bleu_3': 0.0,
  'bleu_4': 0.0},
 'rouge': {'rouge1_fmeasure': 0.0963231697678566,
  'rouge1_precision': 0.09426577389240265,
  'rouge1_recall': 0.14226017892360687,
  'rouge2_fmeasure': 0.022457709535956383,
  'rouge2_precision': 0.023563219234347343,
  'rouge2_recall': 0.03511904925107956,
  'rougeL_fmeasure': 0.0963231697678566,
  'rougeL_precision': 0.09426577389240265,
  'rougeL_recall': 0.14226017892360687,
  'rougeLsum_fmeasure': 0.0963231697678566,
  'rougeLsum_precision': 0.09426577389240265,
  'rougeLsum_recall': 0.14226017892360687},
 'bert': {'bert_score': 0.5274179577827454}}

In [ ]:
# Open a file in write mode
with open('Dolly-result.json', 'w') as json_file:
    # Write the JSON data to the file using json.dump()
    json.dump(results, json_file)